In [2]:
import requests
import json
from pprint import pprint
import pandas as pd
import datetime
from config import api_key

In [20]:
base_url = "https://api.darksky.net/forecast/[key]/[latitude],[longitude]"
test_url = "https://api.darksky.net/forecast/" + api_key + "-83.489545,2009-02-03T12:00:00"

file = "Resources/Top_10_Parks.csv"

park_df = pd.read_csv(file)

park_df.head()

,Park,Lat,Lng
0,Great Smoky Mountains National Park,35.611764,-83.489545
1,Grand Canyon National Park,36.106965,-112.112997
2,Yosemite National Park,37.865101,-119.538329
3,Rocky Mountain National Park,40.342793,-105.683639
4,Zion National Park,37.298202,-113.026300


In [21]:
response = requests.get(test_url)
json_response = response.json()
print(response)

<Response [200]>


In [22]:
pprint(json_response)

{'currently': {'apparentTemperature': 24.76,
               'cloudCover': 1,
               'dewPoint': 17.16,
               'humidity': 0.72,
               'icon': 'cloudy',
               'precipIntensity': 0.0008,
               'precipProbability': 0.22,
               'precipType': 'snow',
               'summary': 'Overcast',
               'temperature': 24.76,
               'time': 1233680400,
               'uvIndex': 3,
               'visibility': 10,
               'windBearing': 250,
               'windGust': 10.62,
               'windSpeed': 2.34},
 'daily': {'data': [{'apparentTemperatureHigh': 24.76,
                     'apparentTemperatureHighTime': 1233680400,
                     'apparentTemperatureLow': -6.2,
                     'apparentTemperatureLowTime': 1233745200,
                     'apparentTemperatureMax': 24.76,
                     'apparentTemperatureMaxTime': 1233680400,
                     'apparentTemperatureMin': -1.54,
                    

In [8]:
# Create range of dates to use in loop when going through API

dates = pd.date_range(start='2009-01-01', end='2018-12-31', freq='3d')

print(dates)

DatetimeIndex(['2009-01-01', '2009-01-04', '2009-01-07', '2009-01-10',
               '2009-01-13', '2009-01-16', '2009-01-19', '2009-01-22',
               '2009-01-25', '2009-01-28',
               ...
               '2018-12-04', '2018-12-07', '2018-12-10', '2018-12-13',
               '2018-12-16', '2018-12-19', '2018-12-22', '2018-12-25',
               '2018-12-28', '2018-12-31'],
              dtype='datetime64[ns]', length=1218, freq='3D')


In [14]:
# Create list of dates in the format needed for the API calls

url_dates = []
reg_dates = []
months = []
years = []

for x in dates:
    date_parts = datetime.datetime.strptime((str(x).split(' ', 1)[0]), "%Y-%m-%d")
    # month number
    months.append(date_parts.month)
    years.append(date_parts.year)
    reg_dates.append((str(x).split(' ', 1)[0]))
    url_dates.append((str(x).split(' ', 1)[0]) + "T12:00:00")
    
print(url_dates)

['2009-01-01T12:00:00', '2009-01-04T12:00:00', '2009-01-07T12:00:00', '2009-01-10T12:00:00', '2009-01-13T12:00:00', '2009-01-16T12:00:00', '2009-01-19T12:00:00', '2009-01-22T12:00:00', '2009-01-25T12:00:00', '2009-01-28T12:00:00', '2009-01-31T12:00:00', '2009-02-03T12:00:00', '2009-02-06T12:00:00', '2009-02-09T12:00:00', '2009-02-12T12:00:00', '2009-02-15T12:00:00', '2009-02-18T12:00:00', '2009-02-21T12:00:00', '2009-02-24T12:00:00', '2009-02-27T12:00:00', '2009-03-02T12:00:00', '2009-03-05T12:00:00', '2009-03-08T12:00:00', '2009-03-11T12:00:00', '2009-03-14T12:00:00', '2009-03-17T12:00:00', '2009-03-20T12:00:00', '2009-03-23T12:00:00', '2009-03-26T12:00:00', '2009-03-29T12:00:00', '2009-04-01T12:00:00', '2009-04-04T12:00:00', '2009-04-07T12:00:00', '2009-04-10T12:00:00', '2009-04-13T12:00:00', '2009-04-16T12:00:00', '2009-04-19T12:00:00', '2009-04-22T12:00:00', '2009-04-25T12:00:00', '2009-04-28T12:00:00', '2009-05-01T12:00:00', '2009-05-04T12:00:00', '2009-05-07T12:00:00', '2009-05-1

In [11]:
# Make dataframe of dates, api dates, years, and months
date_df = pd.DataFrame({"date" : reg_dates,
                        "url_date" : url_dates,
                        "year" : years,
                        "month" : months})

date_df.head()

,date,url_date,year,month
0,2009-01-01,2009-01-01T12:00:00,2009,1
1,2009-01-04,2009-01-04T12:00:00,2009,1
2,2009-01-07,2009-01-07T12:00:00,2009,1
3,2009-01-10,2009-01-10T12:00:00,2009,1
4,2009-01-13,2009-01-13T12:00:00,2009,1


In [24]:

base_url = "https://api.darksky.net/forecast/" + api_key + "/" 

parks = park_df["Park"]
lat = park_df["Lat"]
lng = park_df["Lng"]



# Empty lists for new dataframe
df_dates = []
df_months = []
df_years = []
df_parks = []
temps = []

for x in range(len(parks)):
    coord = str(lat[x]) + "," + str(lng[x])
    
    print("Currently gathering temperatures for " + parks[x])
    
    for z in range(len(url_dates)):
        try:
            weather_url = base_url + coord + "," + url_dates[z]
            response = requests.get(weather_url).json()
            temp = response["daily"]["data"][0]["temperatureHigh"]
            temps.append(temp)
            df_dates.append(reg_dates[z])
            df_months.append(months[z])
            df_years.append(years[z])
            df_parks.append(parks[x])
#           print(parks[x] + " temperature (" + reg_dates[z] + "): " + str(temp))
        except KeyError:
            print("Record not found, skipping.")
        

Currently gathering temperatures for Great Smoky Mountains National Park
Currently gathering temperatures for Grand Canyon National Park
Record not found, skipping.
Currently gathering temperatures for Yosemite National Park
Record not found, skipping.
Currently gathering temperatures for Rocky Mountain National Park
Record not found, skipping.
Currently gathering temperatures for Zion National Park
Record not found, skipping.
Currently gathering temperatures for Yellowstone National Park
Record not found, skipping.
Currently gathering temperatures for Olympic National Park
Record not found, skipping.
Currently gathering temperatures for Acadia National Park
Currently gathering temperatures for Grand Teton National Park
Record not found, skipping.
Currently gathering temperatures for Glacier National Park
Record not found, skipping.


In [25]:
final_df = pd.DataFrame({"Park" : df_parks,
                        "Date" : df_dates,
                        "Year" : df_years,
                        "Month" : df_months,
                        "Temp" : temps})


final_df.head()

,Park,Date,Year,Month,Temp
0,Great Smoky Mountains National Park,2009-01-01,2009,1,39.30
1,Great Smoky Mountains National Park,2009-01-04,2009,1,57.32
2,Great Smoky Mountains National Park,2009-01-07,2009,1,43.46
3,Great Smoky Mountains National Park,2009-01-10,2009,1,38.71
4,Great Smoky Mountains National Park,2009-01-13,2009,1,38.00


In [26]:
final_df.to_csv("final_df.csv", index = False, header = True)

In [33]:
avg_by_month_df = final_df.groupby(["Park", "Year", "Month"]).mean()
avg_by_month_df = avg_by_month_df.reset_index()
avg_by_month_df

#avg_by_month_df.to_csv("avg_by_month.csv", index = False, header = True)

,Park,Year,Month,Temp
0,Acadia National Park,2009,1,21.501818
1,Acadia National Park,2009,2,28.770000
2,Acadia National Park,2009,3,32.956000
3,Acadia National Park,2009,4,45.278000
4,Acadia National Park,2009,5,54.897273
5,Acadia National Park,2009,6,59.150000
6,Acadia National Park,2009,7,62.654000
7,Acadia National Park,2009,8,68.203000
8,Acadia National Park,2009,9,59.403000
9,Acadia National Park,2009,10,48.867273


In [35]:
avg_by_year_df = final_df.groupby(["Park", "Year"]).mean()
del avg_by_year_df["Month"]
avg_by_year_df = avg_by_year_df.reset_index()
avg_by_year_df

avg_by_year_df.to_csv("avg_by_year.csv", index = False, header = True)

In [41]:
avg_by_year_all_df = final_df.groupby(["Year"]).mean()
del avg_by_year_all_df["Month"]
avg_by_year_all_df = avg_by_year_all_df.reset_index()
avg_by_year_all_df

avg_by_year_all_df.to_csv("avg_by_year_all_df.csv", index = False, header = True)

In [43]:
avg_by_year_month_all_df = final_df.groupby(["Year","Month"]).mean()
avg_by_year_month_all_df = avg_by_year_month_all_df.reset_index()
avg_by_year_month_all_df

avg_by_year_month_all_df.to_csv("avg_by_year_month_all_df.csv", index = False, header = True)